In [80]:
import pandas as pd, numpy as np
import regex as re
#global options
pd.set_option('display.max_columns', None)

In [81]:


'''
Outputs an array of dictionaries, with each dictionary representing a request.
@param data_array an array containing each line of the txt file as an item
'''
def process_http_data(data_array, is_anomalous):
    data_processed = []
    i = 0 
    while i < (len(data_array)-1):
        data_item = {}
        if data_array[i][0:3] == 'GET':
            data_item['method']='GET'
            data_item['url']= data_array[i][4:-1]
            i+=1

            while (i<len(data_array)-1) & (data_array[i]!='\n') :
                s = data_array[i].split(':',1)
                data_item[ s[0] ] = s[1][1:-1]
                i += 1
            i +=2

        elif data_array[i][0:4] == 'POST':
            data_item['method']='POST'
            data_item['url']= data_array[i][5:-1]
            i+=1

            while data_array[i]!='\n':
                s = data_array[i].split(':',1)
                data_item[s[0] ] = s[1][1:-1]
                i += 1

            i += 1
            data_item['body'] = data_array[i][0:-1]
            i+= 2

        else:
            i+=1
            continue

        data_item['anomalous'] = is_anomalous
        data_processed.append(data_item)

    return data_processed



In [82]:
with open('../data/raw/normalTrafficTest.txt') as file:
    data1 =  file.readlines()

with open('../data/raw/normalTrafficTraining.txt') as file:
    data2 = file.readlines()

with open('../data/raw/anomalousTrafficTest.txt') as file:
    data3 = file.readlines()

d3 = process_http_data(data3, True)
d2 = process_http_data(data2, False)
d1 = process_http_data(data1, False)

d3[-1]

{'method': 'GET',
 'url': 'http://localhost:8080/tienda1/imagenes/nuestratierra.jpg.BAK HTTP/1.1',
 'User-Agent': 'Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.8 (like Gecko)',
 'Pragma': 'no-cache',
 'Cache-control': 'no-cache',
 'Accept': 'text/xml,application/xml,application/xhtml+xml,text/html;q=0.9,text/plain;q=0.8,image/png,*/*;q=0.5',
 'Accept-Encoding': 'x-gzip, x-deflate, gzip, deflate',
 'Accept-Charset': 'utf-8, utf-8;q=0.5, *;q=0.5',
 'Accept-Language': 'en',
 'Host': 'localhost:8080',
 'Cookie': 'JSESSIONID=3CC12010CDA952F123240EBAD79B55CC',
 'Connection': 'close',
 'anomalous': True}

In [83]:
df = pd.DataFrame(d1 + d2 + d3)
del(d3,d2,d1)
df.head()

,method,url,User-Agent,Pragma,Cache-control,Accept,Accept-Encoding,Accept-Charset,Accept-Language,Host,Cookie,Connection,anomalous,Content-Type,Content-Length,body
0,GET,http://localhost:8080/tienda1/index.jsp HTTP/1.1,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=EA414B3E327DED6875848530C864BD8F,close,False,NaN,NaN,NaN
1,GET,http://localhost:8080/tienda1/publico/anadir.j...,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=54E25FF4B7F0E4E855B112F882E9EEA5,close,False,NaN,NaN,NaN
2,POST,http://localhost:8080/tienda1/publico/anadir.j...,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=788887A0F479749C4CEEA1E268B4A501,close,False,application/x-www-form-urlencoded,74,id=1&nombre=Jam%F3n+Ib%E9rico&precio=39&cantid...
3,GET,http://localhost:8080/tienda1/publico/autentic...,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=94ECD5EE8EF7EFE4BB26C701B150ED7B,close,False,NaN,NaN,NaN
4,POST,http://localhost:8080/tienda1/publico/autentic...,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=23391DBBADEC19FE01E02D201F278C6A,close,False,application/x-www-form-urlencoded,60,modo=entrar&login=caria&pwd=egipciaca&remember...


In [84]:
# process columns, extract features
df['browser'] = df['User-Agent'].str.extract( r'^(.*?) \(', expand=False)
df['system-information'] = df['User-Agent'].str.extract( r'\((.*?)\)', expand=False)
df['platform'] = df['User-Agent'].str.extract( r'\) (.*)$', expand=False)
df.drop('User-Agent',1)


df.head()

,method,url,User-Agent,Pragma,Cache-control,Accept,Accept-Encoding,Accept-Charset,Accept-Language,Host,Cookie,Connection,anomalous,Content-Type,Content-Length,body,browser,system-information,platform
0,GET,http://localhost:8080/tienda1/index.jsp HTTP/1.1,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=EA414B3E327DED6875848530C864BD8F,close,False,NaN,NaN,NaN,Mozilla/5.0,compatible; Konqueror/3.5; Linux,KHTML/3.5.8 (like Gecko)
1,GET,http://localhost:8080/tienda1/publico/anadir.j...,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=54E25FF4B7F0E4E855B112F882E9EEA5,close,False,NaN,NaN,NaN,Mozilla/5.0,compatible; Konqueror/3.5; Linux,KHTML/3.5.8 (like Gecko)
2,POST,http://localhost:8080/tienda1/publico/anadir.j...,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=788887A0F479749C4CEEA1E268B4A501,close,False,application/x-www-form-urlencoded,74,id=1&nombre=Jam%F3n+Ib%E9rico&precio=39&cantid...,Mozilla/5.0,compatible; Konqueror/3.5; Linux,KHTML/3.5.8 (like Gecko)
3,GET,http://localhost:8080/tienda1/publico/autentic...,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=94ECD5EE8EF7EFE4BB26C701B150ED7B,close,False,NaN,NaN,NaN,Mozilla/5.0,compatible; Konqueror/3.5; Linux,KHTML/3.5.8 (like Gecko)
4,POST,http://localhost:8080/tienda1/publico/autentic...,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=23391DBBADEC19FE01E02D201F278C6A,close,False,application/x-www-form-urlencoded,60,modo=entrar&login=caria&pwd=egipciaca&remember...,Mozilla/5.0,compatible; Konqueror/3.5; Linux,KHTML/3.5.8 (like Gecko)


In [85]:
tmp = re.split('[?&]', 'http://localhost:8080/tienda1/publico/anadir.jsp?id=1&nombre=Jam%F3n+Ib%E9rico&precio=39&cantidad=41&B1=A%F1adir+al+carrito HTTP/1.1')
print(tmp)
tmp = re.split('[?/&=+]', 'http://localhost:8080/tienda1/publico/anadir.jsp?id=1&nombre=Jam%F3n+Ib%E9rico&precio=39&cantidad=41&B1=A%F1adir+al+carrito HTTP/1.1')
print(tmp)
tmp = re.split('[&]','modo=registro&login=minthorn&password=ahorquillado&nombre=Isra%E9l&apellidos=Luna+Barberan&email=pouyet%40mcrspain.mh&dni=83073375C&direccion=Calle+Pamplona%2C+26+&ciudad=Arrankudiaga&cp=49740&provincia=Huesca&ntc=2656905158370014&B1=Registrar' )
print(tmp)

['http://localhost:8080/tienda1/publico/anadir.jsp', 'id=1', 'nombre=Jam%F3n+Ib%E9rico', 'precio=39', 'cantidad=41', 'B1=A%F1adir+al+carrito HTTP/1.1']
['http:', '', 'localhost:8080', 'tienda1', 'publico', 'anadir.jsp', 'id', '1', 'nombre', 'Jam%F3n', 'Ib%E9rico', 'precio', '39', 'cantidad', '41', 'B1', 'A%F1adir', 'al', 'carrito HTTP', '1.1']
['modo=registro', 'login=minthorn', 'password=ahorquillado', 'nombre=Isra%E9l', 'apellidos=Luna+Barberan', 'email=pouyet%40mcrspain.mh', 'dni=83073375C', 'direccion=Calle+Pamplona%2C+26+', 'ciudad=Arrankudiaga', 'cp=49740', 'provincia=Huesca', 'ntc=2656905158370014', 'B1=Registrar']


In [86]:
df['protocol'] = df['url'].str.extract(r' (.*?)$')
df['protocol'].value_counts()

HTTP/1.1    96668
Name: protocol, dtype: int64

In [88]:
def extract_url_only(x):
    if '?' in x:
        return x.split('?')[0]
    return x[:-8]

def extract_url_args_only(x):
    if '?' in x:
        return re.split( '[&]', x.split('?')[1] )
    return []    
    
    
df['url_only'] = df['url'].apply(lambda x: extract_url_only(x))
df['url_only']

df['url_args_only'] = df['url'].apply(lambda x: extract_url_args_only(x))
df['url_args_only'] 

0                                                       []
1        [id=1, nombre=Jam%F3n+Ib%E9rico, precio=39, ca...
2                                                       []
3        [modo=entrar, login=caria, pwd=egipciaca, reme...
4                                                       []
                               ...                        
96663    [modo=registro, login=beaumont, password=quEra...
96664                                                   []
96665                                                   []
96666                                                   []
96667                                                   []
Name: url_args_only, Length: 96668, dtype: object

In [89]:
df['body']

0                                                      NaN
1                                                      NaN
2        id=1&nombre=Jam%F3n+Ib%E9rico&precio=39&cantid...
3                                                      NaN
4        modo=entrar&login=caria&pwd=egipciaca&remember...
                               ...                        
96663                                                  NaN
96664    modo=registro&login=beaumont&password=quEratIt...
96665                                                  NaN
96666                                                  NaN
96667                                                  NaN
Name: body, Length: 96668, dtype: object

In [90]:
def extract_body_words(x):
    if type(x)==str:
        return re.split('[&]', x)
    return []
 

df['body_words'] = df['body'].apply( lambda x: extract_body_words(x) )
df['body_words']

0                                                       []
1                                                       []
2        [id=1, nombre=Jam%F3n+Ib%E9rico, precio=39, ca...
3                                                       []
4        [modo=entrar, login=caria, pwd=egipciaca, reme...
                               ...                        
96663                                                   []
96664    [modo=registro, login=beaumont, password=quEra...
96665                                                   []
96666                                                   []
96667                                                   []
Name: body_words, Length: 96668, dtype: object

In [ ]:
## feature characteristics
df['num_of_args']
df['max_length_of_args']
df['length_args']
df['total_length']
df['num_sql_words']
df['num_special_chars']

In [91]:
df.head()

,method,url,User-Agent,Pragma,Cache-control,Accept,Accept-Encoding,Accept-Charset,Accept-Language,Host,Cookie,Connection,anomalous,Content-Type,Content-Length,body,browser,system-information,platform,protocol,req_url,url_only,url_args_only,body_words
0,GET,http://localhost:8080/tienda1/index.jsp HTTP/1.1,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=EA414B3E327DED6875848530C864BD8F,close,False,NaN,NaN,NaN,Mozilla/5.0,compatible; Konqueror/3.5; Linux,KHTML/3.5.8 (like Gecko),HTTP/1.1,NaN,http://localhost:8080/tienda1/index.jsp,[],[]
1,GET,http://localhost:8080/tienda1/publico/anadir.j...,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=54E25FF4B7F0E4E855B112F882E9EEA5,close,False,NaN,NaN,NaN,Mozilla/5.0,compatible; Konqueror/3.5; Linux,KHTML/3.5.8 (like Gecko),HTTP/1.1,http://localhost:8080/tienda1/publico/anadir.jsp,http://localhost:8080/tienda1/publico/anadir.jsp,"[id=1, nombre=Jam%F3n+Ib%E9rico, precio=39, ca...",[]
2,POST,http://localhost:8080/tienda1/publico/anadir.j...,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=788887A0F479749C4CEEA1E268B4A501,close,False,application/x-www-form-urlencoded,74,id=1&nombre=Jam%F3n+Ib%E9rico&precio=39&cantid...,Mozilla/5.0,compatible; Konqueror/3.5; Linux,KHTML/3.5.8 (like Gecko),HTTP/1.1,NaN,http://localhost:8080/tienda1/publico/anadir.jsp,[],"[id=1, nombre=Jam%F3n+Ib%E9rico, precio=39, ca..."
3,GET,http://localhost:8080/tienda1/publico/autentic...,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=94ECD5EE8EF7EFE4BB26C701B150ED7B,close,False,NaN,NaN,NaN,Mozilla/5.0,compatible; Konqueror/3.5; Linux,KHTML/3.5.8 (like Gecko),HTTP/1.1,http://localhost:8080/tienda1/publico/autentic...,http://localhost:8080/tienda1/publico/autentic...,"[modo=entrar, login=caria, pwd=egipciaca, reme...",[]
4,POST,http://localhost:8080/tienda1/publico/autentic...,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=23391DBBADEC19FE01E02D201F278C6A,close,False,application/x-www-form-urlencoded,60,modo=entrar&login=caria&pwd=egipciaca&remember...,Mozilla/5.0,compatible; Konqueror/3.5; Linux,KHTML/3.5.8 (like Gecko),HTTP/1.1,NaN,http://localhost:8080/tienda1/publico/autentic...,[],"[modo=entrar, login=caria, pwd=egipciaca, reme..."
